In [1]:
import os
import sys

root_dir = os.getcwd()
print(root_dir)

d:\NCSI\3 - Thuc nghiem\ModifiedModel\AE_LGBM_2020\mod_AE_LGBM\Human


In [2]:
import pandas as pd
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout

print(root_dir)
os.chdir(r"D:\NCSI\3 - Thuc nghiem\ModifiedModel\AE_LGBM_2020\mod_AE_LGBM\Yeast")


label = pd.read_csv("label.csv")
# print(label)
feat_a = pd.read_csv("total_features_a.csv")
# print(feat_a)
feat_b = pd.read_csv("total_features_b.csv")
# print(feat_b)
feat_all = pd.concat([feat_a, feat_b], axis=1)
print(feat_all.shape)

X_no_enc = feat_all.values
y_no_enc = label.values

d:\NCSI\3 - Thuc nghiem\ModifiedModel\AE_LGBM_2020\mod_AE_LGBM\Human
(8161, 1232)


# Tạo AutoEncoder A, B; huấn luyện và mã hóa

In [3]:
from tensorflow.keras import Model
def AE_net(n_col, k = 208):
    model_AE = Sequential(
        [
            Dense(k, activation = 'sigmoid', input_shape=(n_col,)),
            Dropout(rate = 0.2),
            Dense(n_col, activation = 'sigmoid', input_shape=(k,))
        ]
    )

    # Constructing Autoencoder
    model_AE.compile(optimizer='nadam',
                      loss= 'mean_squared_error',
                      metrics = "accuracy")

    # model_AE1.summary()
    enc_net = Model(inputs=model_AE.input,
                    outputs=model_AE.layers[1].output)
    # enc_net.summary()
    return model_AE, enc_net

In [4]:
#Initialize the Autoencoder for chain A. B
k = 208
n_col = feat_a.shape[1]
model_AE1, enc_a = AE_net(n_col, k)

# Fitting the Autoencoder
print("\nHuan luyen AE 1...")
print(f"feat_a {feat_a.shape}")
history_a = model_AE1.fit(feat_a, feat_a,
                          epochs=200,
                          batch_size=50,
                          validation_split = 0.2,
                          verbose=0)

feat_a_enc = enc_a.predict(feat_a)
print(f"feat_a_enc {feat_a_enc.shape}")


Huan luyen AE 1...
feat_a (8161, 616)


In [ ]:
print("\nHuan luyen AE 2...")
print(f"feat_b {feat_b.shape}")
model_AE2, enc_b = AE_net(n_col, k)
history_b = model_AE2.fit(feat_b, feat_b,
                          epochs=200,
                          batch_size=50,
                          validation_split = 0.2,
                          verbose=0)

feat_b_enc = enc_b.predict(feat_b)
print(feat_b_enc.shape)


Huan luyen AE 2...
feat_b (8161, 616)
(8161, 208)


In [ ]:
import numpy as np

feat_all_enc = np.concatenate((feat_a_enc, feat_b_enc), axis=1)
print(feat_all_enc.shape)

(8161, 416)


In [ ]:
import numpy as np

X_tr = np.copy(feat_all_enc)
y_tr = np.copy(label)
y_tr[y_tr == -1] = 0
print(y_tr)

[[1]
 [1]
 [1]
 ...
 [0]
 [0]
 [0]]


In [ ]:
import lightgbm as lgb

from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.pipeline import Pipeline

gbm = lgb.LGBMClassifier(
    n_estimators=500, # 500
    num_leaves=80,  # 80
    # learning_rate=0.1,  # 0.05
    # max_depth=10,
    #  feature_fraction=0.9,
    # bagging_fraction=0.8,
    # bagging_freq=5,
)

# gbm = lgb.LGBMClassifier(**params)

# pipeline = Pipeline([('transformer', t), ('estimator', gbm)])
cv = StratifiedKFold(n_splits=5, random_state=48, shuffle=True)
print(f'\n{cv.__str__()}...')
scr = cross_val_score(estimator=gbm,
                      X=X_tr, y=y_tr, cv=cv, n_jobs=2)
print(scr)
print(f"mean {np.mean(scr):0.4f}, std {np.std(scr):0.3f}")


StratifiedKFold(n_splits=5, random_state=48, shuffle=True)...
[0.98040416 0.98284314 0.98161765 0.97977941 0.98529412]
mean 0.9820, std 0.002


In [ ]:
import lightgbm as lgb

gbm = lgb.LGBMClassifier(
    n_estimators=500,
    num_leaves=80,
    learning_rate=0.1,  # 0.05
)

gbm = gbm.fit(X_tr, y_tr)

c:\Users\thnha\Anaconda3\envs\TFCONDA26\lib\site-packages\sklearn\preprocessing\_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\thnha\Anaconda3\envs\TFCONDA26\lib\site-packages\sklearn\preprocessing\_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
import os
# load test
os.chdir(r"D:\NCSI\3 - Thuc nghiem\ModifiedModel\AE_LGBM_2020\mod_AE_LGBM\Independent Species\Ecoli")
feat_a = pd.read_csv("total_features_a.csv")
feat_a = enc_a.predict(feat_a)

feat_b = pd.read_csv("total_features_b.csv")
feat_b = enc_b.predict(feat_b)

data_te = np.concatenate((feat_a, feat_b), axis=1)
print('data_te', data_te.shape)
label_te = [1] * data_te.shape[0]

data_te (4013, 416)


In [ ]:
print(gbm.score(data_te, label_te))

0.9982556690755046


In [ ]:
gbm.score(X_tr, y_tr)

1.0